### Step 1: Setup Environment
We need to install the necessary Python libraries and the `ngrok` tool to make our app accessible from the internet.

In [ ]:
!pip install streamlit pyngrok requests pillow

### Step 2: Install & Start AI Backend (Ollama)
We will install **Ollama**, a tool that lets us run powerful AI models locally.

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
import time

# Start Ollama server in the background
print("Starting Ollama Server...")
subprocess.Popen(["ollama", "serve"])
time.sleep(5) # Give it a few seconds to start
print("Ollama is running! 🦙")

### Step 3: Download the AI Model
We will use `llava` (Large Language-and-Vision Assistant), a model that can see images and chat.

In [ ]:
print("Downloading AI Model (this may take a minute)...")
!ollama pull llava
print("Model downloaded! ✅")

### Step 4: Write the App Code
Here is the Python code for our AI Tutor app. 
**Task**: Read through the code below. Can you find where we define the `system_prompt`? Try changing it to give the AI a different personality!

In [ ]:
%%writefile app.py
import streamlit as st
import requests
import base64

# --- Configuration ---
OLLAMA_URL = "http://localhost:11434"
MODEL_NAME = "llava"

# 🎨 UI Setup
st.set_page_config(page_title="My DSE AI Tutor", page_icon="🤖")
st.title("🤖 DSE ICT AI Tutor")
st.caption("Powered by Ollama & Streamlit")

# 🧠 AI Logic
def ask_ai(image_bytes, user_question):
    url = f"{OLLAMA_URL}/api/generate"
    
    # Convert image to format AI understands (Base64)
    img_b64 = base64.b64encode(image_bytes).decode('utf-8')
    
    # Instructions for the AI
    system_prompt = "You are a helpful DSE ICT Tutor. Explain concepts clearly to a secondary school student."
    full_prompt = f"{system_prompt}\n\nQuestion: {user_question}"
    
    payload = {
        "model": MODEL_NAME,
        "prompt": full_prompt,
        "images": [img_b64],
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload)
        return response.json().get("response", "Error: No response")
    except Exception as e:
        return f"Connection Error: {e}"

# 💬 Chat Interface
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display History
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.write(msg["content"])
        if "image" in msg:
            st.image(msg["image"], width=200)

# Input Area
uploaded_file = st.file_uploader("Upload an image (Question/Diagram)", type=["png", "jpg", "jpeg"])
user_input = st.chat_input("Ask a question about the image...")

if user_input and uploaded_file:
    # Show User Message
    with st.chat_message("user"):
        st.write(user_input)
        st.image(uploaded_file, width=200)
    
    # Save to history
    st.session_state.messages.append({
        "role": "user", 
        "content": user_input,
        "image": uploaded_file.getvalue()
    })
    
    # Get AI Response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = ask_ai(uploaded_file.getvalue(), user_input)
            st.write(response)
    
    # Save AI response
    st.session_state.messages.append({"role": "assistant", "content": response})

elif user_input and not uploaded_file:
    st.warning("Please upload an image first! This is a Vision AI demo.")

### Step 5: Launch the App!
Run the cell below. It will generate a link (e.g., `http://xxxx.ngrok-free.app`). Click it to open your app.

In [ ]:
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Create tunnel
public_url = ngrok.connect(8501).public_url
print(f"🚀 Your App is Live! Click here: {public_url}")

# Run the app
!streamlit run app.py